#ref code!!!
<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/keras/restorers/Train_MirNetv2_Restorers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{restorers-mirnetv2-train} -->

ref code: https://colab.research.google.com/github/wandb/examples/blob/restorers/colabs/keras/restorers/Train_MirNetv2_Restorers.ipynb

#Pls read : code Train_MirNetv2
*   เป็นโค้ดที่ดึงค่า  weight  มาจาก Restorers ใน https://github.com/TrystAI/restorers
*   ในกรณีที่คุณต้องการใช้ ค่า  weight จาก original code (มี task เพิ่มความคมชัด,contrast,ลดnoising) สามารถทดลองดึงได้จาก !git clone https://github.com/swz30/MIRNetv2.git หรือ https://colab.research.google.com/drive/1HN9Sd8UEqB1k_O8RpdRLL8ZUKcxh5LP8#scrollTo=GLDZ9t1pm9JZ
*   ในกรณีที่ต้องการเขียนและเทรนโมเดลด้วยตัวเอง สามารถทดลองโค้ด https://colab.research.google.com/drive/1M-ZTWXVHWygfHq4vsFZcGRdh4iJVkGYl?usp=sharing

เนื่องจากเราต้องการทดลองความสามารถของ MirNetv2 จึงไม่ได้มีการปรับ  config เพื่อเพิ่มความสามารถในการเทรนโมเดล




## ดึง  library และ โครงสร้าง model จาก  Github

In [ ]:
!pip install -q --upgrade pip setuptools
!pip install git+https://github.com/soumik12345/restorers.git

  Cloning https://github.com/soumik12345/restorers.git to /tmp/pip-req-build-y53blcvz
  Running command git clone --filter=blob:none --quiet https://github.com/soumik12345/restorers.git /tmp/pip-req-build-y53blcvz
  Resolved https://github.com/soumik12345/restorers.git to commit f84b63af3f6b5fc117e2a34eac00880180f8aa0e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import wandb
import tensorflow as tf
from restorers.dataloader import LOLDataLoader

/usr/local/lib/python3.10/dist-packages/restorers/dataloader/base/base_low_light_dataloader.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## log in wandb + Import dataset
*   สามารถดู wandb_api_key ของตัวเอง ได้ที่ https://wandb.ai/authorize
*   ดึง Lol dataset จาก artifact_address
*   ตั้งเงื่อนไขสำหรับชุดข้อมูล





In [ ]:
wandb.init(project="low-light-enhancement")
data_loader = LOLDataLoader(
    image_size=128,
    bit_depth=8,
    val_split=0.2,
    visualize_on_wandb=True,
    dataset_artifact_address="ml-colabs/dataset/LoL:v0",
)
datasets = data_loader.get_datasets(batch_size=2)
train_dataset, val_dataset = datasets

wandb: Currently logged in as: kontorn9944 (konthorn). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact LoL:v0, 331.95MB. 1003 files... 
wandb:   1003 of 1003 files downloaded.  
Done. 0:0:4.2


Generating visualizations for Train images:   0%|          | 0/388 [00:00<?, ?it/s]

Generating visualizations for Validation images:   0%|          | 0/97 [00:00<?, ?it/s]

Generating visualizations for Test images:   0%|          | 0/15 [00:00<?, ?it/s]

## Import MirNetv2 from restorers
*   ตั้งเงื่อนไขสำหรับ model

In [ ]:
from restorers.model import MirNetv2
model = MirNetv2(
    channels=80,
    channel_factor=1.5,
    num_mrb_blocks=2,
    add_residual_connection=True,
)

In [ ]:
from restorers.losses import CharbonnierLoss
# import Peak Signal-to-Noise Ratio and Structural Similarity metrics,
# implemented as part of restorers
from restorers.metrics import PSNRMetric, SSIMMetric


loss = CharbonnierLoss(
    # a small constant to avoid division by zero
    epsilon=1e-3,
    # type of reduction applied to the loss, it needs to be
    # explicitly specified in case of distributed training
    reduction=tf.keras.losses.Reduction.SUM,
)


optimizer = tf.keras.optimizers.experimental.AdamW(learning_rate=2e-4,)

psnr_metric = PSNRMetric(max_val=1.0) # peak signal-to-noise ratio metric
ssim_metric = SSIMMetric(max_val=1.0) # structural similarity metric

model.compile(
    optimizer=optimizer, loss=loss, metrics=[psnr_metric, ssim_metric]
)

##Train model + call back  ผลลัพธ์จากการฝึก

In [ ]:
# import the wandb callbacks for keras
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint


callbacks = [
    # define the metrics logger callback;
    # we set the `log_freq="batch"` explicitly
    # to the metrics are logged both batch-wise and epoch-wise
    WandbMetricsLogger(log_freq="batch"),
    # define the model checkpoint callback
    WandbModelCheckpoint(
        filepath="checkpoint",
        monitor="val_loss",
        save_best_only=False,
        save_weights_only=False,
        initial_value_threshold=None,
    )
]

# call model.fit()
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=callbacks,
)

Epoch 1/100
194/194 [==============================] - ETA: 0s - loss: 0.0204 - psnr_metric: 18.2739 - ssim_metric: 0.4857

wandb: Adding directory to artifact (./checkpoint)... Done. 0.1s


194/194 [==============================] - 252s 658ms/step - loss: 0.0204 - psnr_metric: 18.2739 - ssim_metric: 0.4857 - val_loss: 0.3384 - val_psnr_metric: 13.6274 - val_ssim_metric: 0.5495
Epoch 2/100
194/194 [==============================] - ETA: 0s - loss: 0.0151 - psnr_metric: 22.0442 - ssim_metric: 0.5813

wandb: Adding directory to artifact (./checkpoint)... Done. 0.1s


194/194 [==============================] - 120s 618ms/step - loss: 0.0151 - psnr_metric: 22.0442 - ssim_metric: 0.5813 - val_loss: 0.3399 - val_psnr_metric: 13.1527 - val_ssim_metric: 0.5250
Epoch 3/100
194/194 [==============================] - ETA: 0s - loss: 0.0142 - psnr_metric: 23.7493 - ssim_metric: 0.6219

...

194/194 [==============================] - 119s 616ms/step - loss: 0.0145 - psnr_metric: 36.4690 - ssim_metric: 0.6948 - val_loss: 0.3387 - val_psnr_metric: 11.7816 - val_ssim_metric: 0.4095
Epoch 99/100
194/194 [==============================] - ETA: 0s - loss: 0.0145 - psnr_metric: 36.8263 - ssim_metric: 0.7028

wandb: Adding directory to artifact (./checkpoint)... Done. 0.1s


194/194 [==============================] - 123s 635ms/step - loss: 0.0145 - psnr_metric: 36.8263 - ssim_metric: 0.7028 - val_loss: 0.3387 - val_psnr_metric: 11.7901 - val_ssim_metric: 0.4107
Epoch 100/100
194/194 [==============================] - ETA: 0s - loss: 0.0140 - psnr_metric: 36.6460 - ssim_metric: 0.6924

wandb: Adding directory to artifact (./checkpoint)... Done. 0.1s


194/194 [==============================] - 122s 627ms/step - loss: 0.0140 - psnr_metric: 36.6460 - ssim_metric: 0.6924 - val_loss: 0.3387 - val_psnr_metric: 11.7816 - val_ssim_metric: 0.4091


In [ ]:
wandb.finish()

batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,▄▇▄▇█▄▆▆▁▇▁▆▃▇▆▃▇▆▂▇▄▇▆▂▇▃▇▇▄▇▃▇▅▁▇▅▃▇▂▇
batch/psnr_metric,▁▁▂▂▂▃▃▄▄▄▅▅▅▅▅▆▅▆▆▆▇▆▆▆▅▆▆▆▆▆▇▇▇█▆▇█▆▇▆
batch/ssim_metric,▁▃▄▃▃▄▅▅▅▅▇▆▆▅▆▆▆▇▆▆█▆▆▇▅▇▆▆▆▆▇▇▇█▆▆▇▅▆▆
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▂▃▄▃▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▁▂▃▂▁▁▂▁▂▂▃▂▁▂▂▃▂▁▂
epoch/psnr_metric,▁▃▄▄▃▅▅▅▅▆▆▆▅▆▆▇▆▇▇▇▇▇▇▇▇█▇▇▇▇███████▇██
epoch/ssim_metric,▁▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████▇████▇█▇████████
epoch/val_loss,▇▆▇▇▁▅█▂▅█▅▇███▇███▅▆▇▆▆▇█▇█████▇▇▇▇▇▇██
